<a href="https://colab.research.google.com/github/rushikeshnakhate/PortfoliOpt/blob/master/multi_dates_post_processing_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
import yfinance as yf
import pandas as pd
import numpy as np
from pathlib import Path
import os
output_dir = Path("/content/drive/MyDrive/")
data_dir = Path("/content/drive/MyDrive/data")
import matplotlib.pyplot as plt
# !pip install tqdm
# !pip install PyPortfolioOpt
# !pip install tabulate
# !pip install dtale
# !pip install -q data-table
import scipy.optimize as sci_opt
from tqdm import tqdm  # Import the tqdm progress b
from google.colab import data_table

# !rm -rf output_dir/MyDrive/data
# !rm -rf output_dir/MyDrive/data*

In [ ]:
import tarfile

def unzip_files():
  from google.colab import files
  import zipfile

  # After uploading, you can unzip the file
  import zipfile
  import os


  zip_filename = '/content/drive/MyDrive/data.zip'
  # Replace 'data.zip' with the name of your uploaded zip file
  with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
      zip_ref.extractall(output_dir)

  tar_filename = '/content/drive/MyDrive/data.tar.gz'
  # with tarfile.open(tar_filename, 'r:gz') as tar_ref:
  #   tar_ref.extractall(output_dir)

  print("Directory uploaded and extracted successfully to Google Drive.")
unzip_files()

In [ ]:
from datetime import datetime, timedelta
def generate_month_date_ranges(year, months=None):
    """Generate start_date and end_date (without time) for each month in the specified year.

    Args:
        year (int): The year for which to generate the month ranges.
        months (list of int, optional): Specific months to generate ranges for (1-12).

    Returns:
        list of tuples: Each tuple contains the start and end date for the specified months.
    """
    if months is None:
        months = range(1, 13)  # Default to all months if none specified

    month_ranges = []
    for month in months:
        start_date = datetime(year, month, 1).date()
        if month == 12:
            end_date = (datetime(year + 1, 1, 1) - timedelta(days=1)).date()
        else:
            end_date = (datetime(year, month + 1, 1) - timedelta(days=1)).date()
        month_ranges.append((start_date, end_date))

    return month_ranges

def create_current_month_directory(start_date, output_dir):
    current_month = start_date.strftime("%Y%m")
    current_month_dir = output_dir / current_month
    # current_month_dir.rmdir(parents=True, exist_ok=True)
    return current_month_dir

In [ ]:
from google.colab import data_table

# Assuming df is your DataFrame
def clean_metadata(value):
    if isinstance(value, pd.Series):
        return value.values  # Extract just the values
    elif isinstance(value, list):
        return [str(v) for v in value]  # Convert each element to string for clarity
    return value

def extract_value(value):
    # Check if the value is a list and has only one element, then extract it
    # print(type(value))
    if isinstance(value, (np.ndarray, list)) and len(value) == 1:
        return value[0]
    # elif isinstance(value, tuple) and len(value) >= 2:
    #   return value[1]
    return value  # Return the value as-is if not a list



In [ ]:
def load_data_and_plot(start_date, data_dir, plot_func):
    """
    Function to load data for a specific date range and plot using the given plot function.

    Args:
    start_date (datetime): The start date of the month for data loading.
    data_dir (str): The directory where the data is stored.
    plot_func (function): A function to plot the data (either bar plot or 3D plot).
    """
    current_month_dir = create_current_month_directory(start_date, data_dir)
    data_pkl_filepath = os.path.join(current_month_dir, "data.pkl")
    expected_return_pkl_filepath = os.path.join(current_month_dir, "expected_return.pkl")
    optimization_pkl_filepath = os.path.join(current_month_dir, "optimization.pkl")
    post_processing_weight_pkl_filepath = os.path.join(current_month_dir, "post_processing_weight.pkl")


    # Load data
    data = pd.read_pickle(data_pkl_filepath)
    expected_return_df = pd.read_pickle(expected_return_pkl_filepath)
    optimization_df = pd.read_pickle(optimization_pkl_filepath)
    post_processing_df = pd.read_pickle(post_processing_weight_pkl_filepath)


    # Apply this function to every cell in the DataFrame
    df1 = post_processing_df.applymap(clean_metadata)
    optimized_df_cleaned = df1.applymap(extract_value)

    display(post_processing_df)
    # Format the month as 'YYYY-MM' for the title
    current_month = start_date.strftime('%Y-%m')
    # Call the provided plot function
    # print("using optimization_pkl_filepath=optimization_pkl_filepath{}".format(optimization_pkl_filepath))
    return plot_func(optimized_df_cleaned)

In [ ]:
# Update the ranking functions to return the top 10 for best and worst portfolios
target_columns = ['Sharpe Ratio','Annual Volatility','Expected Return Type', 'Risk Model', 'Optimizer', 'Expected Annual Return', 'Weights','Allocation_GreedyPortfolio_weight','Allocation_LpPortfolio_weight','Allocation_GreedyPortfolio_remaining_amount','Allocation_LpPortfolio_remaining_amount']
# target_columns = ['Sharpe Ratio','Annual Volatility','Expected Return Type', 'Risk Model', 'Optimizer', 'Expected Annual Return',  'Weights','Allocation_GreedyPortfolio_weight','Allocation_LpPortfolio_weight']
# Function to rank portfolios based on 'Expected Annual Return', 'GreedyPortfolio_Return', and 'LpPortfolio_Return'
def rank_by_annual_return(df1):
    df = df1.copy()
    # Sort by 'Expected Annual Return'
    df_ranked = df[target_columns]
    best_portfolio_return = df_ranked.nlargest(10, 'Expected Annual Return')  # Top 10 highest return
    worst_portfolio_return = df_ranked.nsmallest(10, 'Expected Annual Return')  # Top 10 lowest return

    return best_portfolio_return, worst_portfolio_return

# Function to rank portfolios based on 'Annual Volatility', 'GreedyPortfolio_Volatility', and 'LpPortfolio_Volatility'
def rank_by_volatility(df1):
    df = df1.copy()
    # Sort by 'Annual Volatility'
    df_ranked = df[target_columns]
    best_portfolio_vol = df_ranked.nsmallest(10, 'Annual Volatility')  # Top 10 smallest volatility (best)
    worst_portfolio_vol = df_ranked.nlargest(10, 'Annual Volatility')  # Top 10 largest volatility (worst)

    return best_portfolio_vol, worst_portfolio_vol

# Function to rank portfolios based on 'Sharpe Ratio', 'GreedyPortfolio_Sharpe', and 'LpPortfolio_Sharpe'
def rank_by_sharpe_ratio(df1):
    df = df1.copy()
    # Sort by 'Sharpe Ratio'
    df_ranked = df[target_columns]
    best_portfolio_sharpe = df_ranked.nlargest(10, 'Sharpe Ratio')  # Top 10 highest Sharpe ratios (best)
    worst_portfolio_sharpe = df_ranked.nsmallest(10, 'Sharpe Ratio')  # Top 10 lowest Sharpe ratios (worst)

    return best_portfolio_sharpe, worst_portfolio_sharpe

# # Assuming filtered_df is already defined, executing the updated functions
# best_return, worst_return = rank_by_annual_return(filtered_df)
# best_volatility, worst_volatility = rank_by_volatility(filtered_df)
# best_sharpe, worst_sharpe = rank_by_sharpe_ratio(filtered_df)

# best_return, worst_return, best_volatility, worst_volatility, best_sharpe, worst_sharpe


# # # Executing the functions
# best_return, worst_return = rank_by_annual_return(filtered_df)
# best_volatility, worst_volatility = rank_by_volatility(filtered_df)
# best_sharpe, worst_sharpe = rank_by_sharpe_ratio(filtered_df)

In [ ]:
if __name__ == "__main__":
    year = 2023
    month_ranges = generate_month_date_ranges(year)
    for start_date, end_date in month_ranges:
      top_best, bottom_worst  = load_data_and_plot(start_date, data_dir, rank_by_annual_return)
      data_table.DataTable(top_best)

In [9]:
# data_pkl_filepath = os.path.join(data_dir, "202301/data.pkl")
# data_df =  pd.read_pickle(data_pkl_filepath)
# data_table.DataTable(data_df)

Ticker,ASIANPAINT.NS,BAJAJFINSV.NS,BAJFINANCE.NS,BHARTIARTL.NS,BPCL.NS,CIPLA.NS,DIVISLAB.NS,GRASIM.NS,HCLTECH.NS,HDFCBANK.NS,HDFCLIFE.NS,ICICIBANK.NS,INFY.NS,JSWSTEEL.NS,KOTAKBANK.NS,NTPC.NS,ONGC.NS,RELIANCE.NS,TATASTEEL.NS,TITAN.NS
Date,,,,,,,,,,,,,,,,,,,,
2023-01-02 00:00:00+00:00,2992.859131,1550.703857,6493.421875,805.292603,152.743790,1044.757935,3298.538086,1708.456299,966.548035,1588.541748,566.499268,880.165771,1463.771729,766.047241,1821.552124,157.131805,138.774216,2361.172119,113.200180,2540.449951
2023-01-03 00:00:00+00:00,2974.198242,1561.191772,6545.438477,809.351135,154.116379,1049.587036,3318.440918,1694.394287,968.314758,1598.929077,591.630615,879.092896,1462.379028,758.735596,1829.337036,156.757706,137.944061,2343.893311,112.440758,2587.828369
2023-01-04 00:00:00+00:00,2963.001709,1546.059204,6510.513184,803.609741,153.864731,1040.807007,3360.055908,1667.997192,963.897888,1570.351685,594.610596,876.898438,1435.725708,727.512695,1817.260376,155.822388,134.900146,2308.602783,109.877739,2571.936523
2023-01-05 00:00:00+00:00,2950.724854,1467.150635,6043.897949,798.957092,156.632721,1061.732300,3385.778076,1675.891724,970.871826,1560.256714,605.388367,857.441101,1416.900269,741.740906,1816.811279,158.908890,135.130753,2304.478027,110.637154,2536.836182
2023-01-06 00:00:00+00:00,2925.237793,1428.695190,5925.349609,787.969177,157.707916,1050.025879,3359.419922,1658.425293,960.364502,1555.087524,607.573730,848.614563,1391.255493,719.657593,1783.375366,158.067123,135.914795,2325.423096,109.782814,2513.072754
2023-01-09 00:00:00+00:00,2930.885254,1412.364258,5938.180664,811.133118,158.645828,1060.854248,3383.479736,1651.517700,992.584290,1558.110962,604.395020,851.784302,1425.784790,733.490479,1808.427246,159.376541,136.652679,2380.329590,111.586426,2460.149658
2023-01-10 00:00:00+00:00,2907.510254,1411.665039,5849.801270,784.850891,158.920334,1052.318237,3424.899170,1629.215942,989.236816,1529.630981,596.200012,840.080566,1411.665649,736.306519,1792.108521,157.085052,137.759583,2345.130859,112.820465,2443.862061
2023-01-11 00:00:00+00:00,2887.916016,1398.080566,5841.478516,757.826355,161.276581,1021.979004,3336.485352,1613.476318,980.821411,1551.673706,595.852295,844.908386,1413.106323,745.199097,1798.396484,155.307953,135.407471,2315.569336,112.820465,2413.365479
2023-01-12 00:00:00+00:00,2863.902588,1386.194458,5844.104492,749.263672,157.868042,1035.197632,3232.668213,1620.729370,996.489685,1559.964233,594.411987,839.397949,1422.086914,754.338684,1776.089355,155.214432,134.807922,2265.566406,112.108513,2422.474854


In [10]:
# data_table.DataTable(performance_df_cleaned.head(10))

In [11]:
# data_table.DataTable(filtered_df)

In [12]:
# print(filtered_df.iloc[1137])